# На участках автомобильных дорог, оборудованных и знаком ограничения скорости, и знаком о работе камер фото- и видеофиксации, регистрируется меньшее число ДТП, чем на участках автомобильных дорог, оборудованных только знаками ограничения скорости

# Загрузим библиотеки и данные

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
!pip install catboost
from catboost import CatBoostClassifier
!pip install geopandas
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import numpy as np
import folium
import geopandas as gpd
import pandas as pd
import json
import pandas as pd
import urllib.request
from math import radians, sin
from sklearn.tree import export_graphviz
!pip install dtreeviz
import graphviz
import dtreeviz

**Данные о ДТП**

In [2]:

#Moskow
url = "https://cms.dtp-stat.ru/media/opendata/moskva.geojson"

with urllib.request.urlopen(url) as url:
    data = json.loads(url.read().decode())

df = pd.json_normalize(data['features'])

In [3]:
df

,type,geometry.type,geometry.coordinates,properties.id,properties.tags,properties.light,properties.point.lat,properties.point.long,properties.nearby,properties.region,...,properties.datetime,properties.severity,properties.vehicles,properties.dead_count,properties.participants,properties.injured_count,properties.parent_region,properties.road_conditions,properties.participants_count,properties.participant_categories
0,Feature,Point,"[37.601433, 55.743458]",1578215,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено",55.743458,37.601433,"[Многоквартирные жилые дома, Нерегулируемый пе...",Хамовники,...,2016-08-30 20:40:00,С погибшими,"[{'year': 2015, 'brand': 'BMW', 'color': 'Крас...",1,"[{'role': 'Пешеход', 'gender': 'Женский', 'vio...",0,Москва,"[Сухое, Отсутствие дорожных знаков в необходим...",2,"[Пешеходы, Мотоциклисты, Все участники]"
1,Feature,Point,"[37.58156, 55.76969]",1578237,[Дорожно-транспортные происшествия],Светлое время суток,55.769690,37.581560,"[Многоквартирные жилые дома, Регулируемый пере...",Пресненский,...,2016-08-11 13:55:00,Легкий,"[{'year': None, 'brand': None, 'color': 'Иные ...",0,[],1,Москва,[Сухое],2,"[Велосипедисты, Все участники]"
2,Feature,Point,"[37.611988, 55.754358]",1578234,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено",55.754358,37.611988,[Регулируемый перекрёсток],Пресненский,...,2016-08-12 21:45:00,Легкий,"[{'year': 2015, 'brand': 'SKODA', 'color': 'Бе...",0,[],1,Москва,[Сухое],2,[Все участники]
3,Feature,Point,"[37.5831, 55.0372]",1578239,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено",55.037200,37.583100,"[Многоквартирные жилые дома, Остановка обществ...",Мещанский,...,2016-08-29 00:05:00,Тяжёлый,"[{'year': 2008, 'brand': 'CITROEN', 'color': '...",0,"[{'role': 'Пешеход', 'gender': 'Мужской', 'vio...",1,Москва,[Сухое],2,"[Пешеходы, Все участники]"
4,Feature,Point,"[37.6986, 55.7364]",1578252,[Дорожно-транспортные происшествия],Светлое время суток,55.736400,37.698600,"[Мост, эстакада, путепровод, Эстакада, путепро...",Таганский,...,2016-08-04 18:00:00,Легкий,"[{'year': 2000, 'brand': 'KAWASAKI', 'color': ...",0,[],1,Москва,[Сухое],2,"[Мотоциклисты, Все участники]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72117,Feature,Point,"[37.500876, 55.698643]",2595220,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено",55.698643,37.500876,"[Многоквартирные жилые дома, Административные ...",Раменки,...,2021-07-13 00:29:00,Легкий,"[{'year': 2019, 'brand': 'VOLKSWAGEN', 'color'...",0,[],1,Москва,[Сухое],3,[Все участники]
72118,Feature,Point,"[37.410818, 55.764557]",2595215,[Дорожно-транспортные происшествия],Светлое время суток,55.764557,37.410818,"[Многоквартирные жилые дома, Выезд с прилегающ...",Крылатское,...,2021-07-12 15:33:00,Легкий,"[{'year': 2014, 'brand': 'INFINITI', 'color': ...",0,"[{'role': 'Пешеход', 'gender': 'Женский', 'vio...",1,Москва,[Сухое],2,"[Пешеходы, Все участники]"
72119,Feature,Point,"[37.670349, 55.781491]",2595213,"[Дорожно-транспортные происшествия, ДТП и пост...",Светлое время суток,55.781491,37.670349,"[Многоквартирные жилые дома, Административные ...",Красносельский,...,2021-07-12 11:25:00,Тяжёлый,"[{'year': 2017, 'brand': 'KIA', 'color': 'Белы...",0,"[{'role': 'Пешеход', 'gender': 'Женский', 'vio...",1,Москва,[Сухое],2,"[Пешеходы, Все участники]"
72120,Feature,Point,"[37.591749, 55.576744]",2595219,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено",55.576744,37.591749,"[Остановка общественного транспорта, Надземный...",Чертаново Южное,...,2021-07-13 00:18:00,С погибшими,"[{'year': 2008, 'brand': 'HONDA', 'color': 'Си...",1,"[{'role': 'Пешеход', 'gender': 'Мужской', 'vio...",0,Москва,[Сухое],2,"[Пешеходы, Все участники]"


**Обработаем данные**

In [ ]:
# Функция для извлечения значений координат
def extract_coordinates(row):
    lon, lat = row['geometry.coordinates']
    return pd.Series({'longitude': lon, 'latitude': lat})

# Применение функции к столбцу 'geometry.coordinates'
df[['longitude', 'latitude']] = df.apply(extract_coordinates, axis=1)


In [ ]:
df['properties.datetime'] = df['properties.datetime'].astype('datetime64')
df['year'] = df['properties.datetime'].dt.year

In [ ]:
#df = df[df['year'] == 2023]
df = df[(df['year'] == 2022) | (df['year'] == 2023)]
# df = df[(df['year'] == 2021) | (df['year'] == 2022) | (df['year'] == 2023)]

In [ ]:
road_znak = pd.read_csv ('data-115269-2022-12-23.csv', sep=';', encoding='cp1251')
road_znak = road_znak.rename(columns=road_znak.iloc[0])
road_znak.reset_index(drop=True, inplace=True)
road_znak = road_znak.drop(0, axis=0).reset_index(drop=True)
head_features = ['8.23 Фотовидеофиксация', '3.25 Конец ограничения максимальной скорости', 'Ограничение максимальной скорости']
road_znak_head = road_znak.loc[(road_znak['Тип дорожного знака'] == '8.23 Фотовидеофиксация') |
       (road_znak['Тип дорожного знака'] == '3.24 Ограничение максимальной скорости')]

lat_fvf = road_znak_head[road_znak_head['Тип дорожного знака'] == '8.23 Фотовидеофиксация']['Широта в WGS-84']
lon_fvf = road_znak_head[road_znak_head['Тип дорожного знака'] == '8.23 Фотовидеофиксация']['Долгота в WGS-84']
road_znak_head_fvf = road_znak_head[road_znak_head['Тип дорожного знака'] == '8.23 Фотовидеофиксация']

lat_speed_limit = road_znak_head[road_znak_head['Тип дорожного знака'] == '3.24 Ограничение максимальной скорости']['Широта в WGS-84']
lon_speed_limit = road_znak_head[road_znak_head['Тип дорожного знака'] == '3.24 Ограничение максимальной скорости']['Долгота в WGS-84']
speed_limit = road_znak_head[road_znak_head['Тип дорожного знака'] == '3.24 Ограничение максимальной скорости']

road_znak_head

In [ ]:
df = df.dropna()

# First_road

**Загрузим координаты первой дороги**

In [ ]:
df_coords = pd.read_json('one_road.json')
distance_kut_pr = df_coords['features'][0]['properties']['segments'][0]['distance']
df_coords = df_coords['features'][0]['geometry']['coordinates']

lons = [coord[0] for coord in df_coords]
lats = [coord[1] for coord in df_coords]

df_coords = pd.DataFrame({'lon': lons, 'lat': lats})
print(df_coords)


In [ ]:
import geopy
import geopy.distance

**Подготовка данных для проверки гипотезы**

В нижеуказанном коде происходит обработка геоданных (координат местоположения) и их визуализация на карте.
- Сначала создаются геодатафреймы (gdf) для центроидов, данных об объектах и мест на карте (df_geometry_gdf, kfvf_gdf и speed_limit_gdf).
- Затем используется модель KMeans для кластеризации центроидов по расстоянию между ними, после чего определяются ближайшие точки к центроидам и получаются сами центроиды.
- Далее создается карта, на которой отображаются красные квадраты, соответствующие центроидам, а также синие точки (дтп), оранжевые точки (КФВФ) и зеленые точки (ограничение скорости), которые находятся внутри красных квадратов.
- Также происходит расчет расстояний между точками в метрах и площади прямоугольника.

Данные действия производятся для возможности тестирования гипотезы разными методами

In [ ]:
# Создаем геодатафрейм для создания центроидов
geometry = gpd.points_from_xy(df_coords.lon, df_coords.lat)
gdf = gpd.GeoDataFrame(df_coords, geometry=geometry)

# Создаем геодатафрейм для df
df_geometry = gpd.points_from_xy(df.longitude, df.latitude)
df_geometry_gdf = gpd.GeoDataFrame(df, geometry=df_geometry)

# Создаем геодатафрейм для КФВФ
kfvf_geometry = gpd.points_from_xy(lon_fvf, lat_fvf)
kfvf_gdf = gpd.GeoDataFrame(road_znak_head_fvf, geometry=kfvf_geometry)

# Создаем геодатафрейм для ограничения скорости
speed_limit_geometry = gpd.points_from_xy(lon_speed_limit, lat_speed_limit)
speed_limit_gdf = gpd.GeoDataFrame(speed_limit, geometry=speed_limit_geometry)


# Определяем количество кластеров для создания
# количество кластеров зависит от расстояния между центроидами,
# задаем минимальное расстояние между центроидами равным 1 км
# коэффициент преобразования градусов в километры примерно равен 111
k = 150

# Создаем модель KMeans
kmeans = KMeans(n_clusters=k)

# Обучаем модель на координатах
kmeans.fit(gdf[['lon', 'lat']])

# Находим ближайшие точки к центроидам
closest_points, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, gdf[['lon', 'lat']])

# Получаем центроиды
centroids = gdf.iloc[closest_points]



# Создаем карту
m = folium.Map(location=[55.7522, 37.6156], zoom_start=12)
q = []
# Добавляем красные квадраты на карту
for _, row in centroids.iterrows():
    # Определяем координаты квадрата, в котором будем искать точки
    xmin, ymin, xmax, ymax = row.lon - (0.02 / 4), row.lat - (0.005 / 4), row.lon + (0.02 / 4), row.lat + (0.005 / 4)
    rectangle = [[ymin, xmin], [ymin, xmax], [ymax, xmax], [ymax, xmin], [ymin, xmin]]


    # Выбираем дтп
    points = df_geometry_gdf.cx[xmin:xmax, ymin:ymax]

    # Выбираем кфвф
    kfvf_points = kfvf_gdf.cx[xmin:xmax, ymin:ymax]

    # Выбираем ограничение скорости
    speed_limit_points = speed_limit_gdf.cx[xmin:xmax, ymin:ymax]


    # Подсчитываем количество точек дтп
    num_points = points.shape[0]

    # Подсчитываем количество КФВФ
    num_kfvf_points = kfvf_points.shape[0]

    # Подсчитываем количество ограничений скорости
    num_speed_limit_points = speed_limit_points.shape[0]

    q.append([num_points, num_kfvf_points, num_speed_limit_points])

    
    folium.Polygon(locations=rectangle, color='red').add_to(m)

# Отображаем синие точки (дтп) на карте
for _, row in df.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']], radius=2, color='blue', fill=True).add_to(m)

# Отображаем оранжевые точки (КФВФ) на карте, которые попадают в красный квадрат
for _, row in road_znak_head_fvf.iterrows():
    folium.CircleMarker(location=[row['Широта в WGS-84'], row['Долгота в WGS-84']], radius=2, color='orange', fill=True).add_to(m)

# Отображаем зеленые точки (ограничение скорости) на карте, которые попадают в красный квадрат
for _, row in speed_limit.iterrows():
    folium.CircleMarker(location=[row['Широта в WGS-84'], row['Долгота в WGS-84']], radius=2, color='green', fill=True).add_to(m)
    
# расчет расстояний между точками в метрах
dist_lat = geopy.distance.distance((ymin, xmin), (ymax, xmin)).meters
dist_lon = geopy.distance.distance((ymin, xmin), (ymin, xmax)).meters

# расчет площади прямоугольника
R = 6371  # радиус Земли в км
area = R ** 2 * abs(sin(radians(ymax)) - sin(radians(ymin))) * radians(xmax - xmin)

print("Площадь прямоугольника:", round(area, 2), "км^2")
print("Ширина прямоугольника:", round(dist_lon / 1000, 2), "км")
print("Высота прямоугольника:", round(dist_lat / 1000, 2), "км")


Синий цвет - ДТП

Оранжевый - КФВФ

Зеленый - знак ограничения скорости

In [ ]:
m

In [ ]:
kut_pros = pd.DataFrame(data=q, columns=['количество_дтп', 'количество_знаков_кфвф', 'количество_знаков_ограничения_скорости'])

# Second_road

**Загрузим данные второй дороги**

In [ ]:
df_coords = pd.read_json('first_road.json')
distance_street_strom = df_coords['features'][0]['properties']['segments'][0]['distance']
df_coords = df_coords['features'][0]['geometry']['coordinates']

lons = [coord[0] for coord in df_coords]
lats = [coord[1] for coord in df_coords]

df_coords = pd.DataFrame({'lon': lons, 'lat': lats})
print(df_coords)


In [ ]:
# Создаем геодатафрейм для создания центроидов
geometry = gpd.points_from_xy(df_coords.lon, df_coords.lat)
gdf = gpd.GeoDataFrame(df_coords, geometry=geometry)

# Создаем геодатафрейм для df
df_geometry = gpd.points_from_xy(df.longitude, df.latitude)
df_geometry_gdf = gpd.GeoDataFrame(df, geometry=df_geometry)

# Создаем геодатафрейм для КФВФ
kfvf_geometry = gpd.points_from_xy(lon_fvf, lat_fvf)
kfvf_gdf = gpd.GeoDataFrame(road_znak_head_fvf, geometry=kfvf_geometry)

# Создаем геодатафрейм для ограничения скорости
speed_limit_geometry = gpd.points_from_xy(lon_speed_limit, lat_speed_limit)
speed_limit_gdf = gpd.GeoDataFrame(speed_limit, geometry=speed_limit_geometry)


# Определяем количество кластеров для создания
# количество кластеров зависит от расстояния между центроидами,
# задаем минимальное расстояние между центроидами равным 1 км
# коэффициент преобразования градусов в километры примерно равен 111
k = 150
# Создаем модель KMeans
kmeans = KMeans(n_clusters=k)

# Обучаем модель на координатах
kmeans.fit(gdf[['lon', 'lat']])

# Находим ближайшие точки к центроидам
closest_points, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, gdf[['lon', 'lat']])

# Получаем центроиды
centroids = gdf.iloc[closest_points]



# Создаем карту
m = folium.Map(location=[55.7522, 37.6156], zoom_start=12)
q = []
# Добавляем красные квадраты на карту
for _, row in centroids.iterrows():
    # Определяем координаты квадрата, в котором будем искать точки
    xmin, ymin, xmax, ymax = row.lon - (0.02 / 4), row.lat - (0.005 / 4), row.lon + (0.02 / 4), row.lat + (0.005 / 4)
    rectangle = [[ymin, xmin], [ymin, xmax], [ymax, xmax], [ymax, xmin], [ymin, xmin]]


    # Выбираем дтп
    points = df_geometry_gdf.cx[xmin:xmax, ymin:ymax]

    # Выбираем кфвф
    kfvf_points = kfvf_gdf.cx[xmin:xmax, ymin:ymax]

    # Выбираем ограничение скорости
    speed_limit_points = speed_limit_gdf.cx[xmin:xmax, ymin:ymax]


    # Подсчитываем количество точек дтп
    num_points = points.shape[0]

    # Подсчитываем количество КФВФ
    num_kfvf_points = kfvf_points.shape[0]

    # Подсчитываем количество ограничений скорости
    num_speed_limit_points = speed_limit_points.shape[0]

    q.append([num_points, num_kfvf_points, num_speed_limit_points])

    
    folium.Polygon(locations=rectangle, color='red').add_to(m)

# Отображаем синие точки (дтп) на карте
for _, row in df.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']], radius=2, color='blue', fill=True).add_to(m)

# Отображаем оранжевые точки (КФВФ) на карте, которые попадают в красный квадрат
for _, row in road_znak_head_fvf.iterrows():
    folium.CircleMarker(location=[row['Широта в WGS-84'], row['Долгота в WGS-84']], radius=2, color='orange', fill=True).add_to(m)

# Отображаем зеленые точки (ограничение скорости) на карте, которые попадают в красный квадрат
for _, row in speed_limit.iterrows():
    folium.CircleMarker(location=[row['Широта в WGS-84'], row['Долгота в WGS-84']], radius=2, color='green', fill=True).add_to(m)

# расчет расстояний между точками в метрах
dist_lat = geopy.distance.distance((ymin, xmin), (ymax, xmin)).meters
dist_lon = geopy.distance.distance((ymin, xmin), (ymin, xmax)).meters

# расчет площади прямоугольника
R = 6371  # радиус Земли в км
area = R ** 2 * abs(sin(radians(ymax)) - sin(radians(ymin))) * radians(xmax - xmin)

print("Площадь прямоугольника:", round(area, 2), "км^2")
print("Ширина прямоугольника:", round(dist_lon / 1000, 2), "км")
print("Высота прямоугольника:", round(dist_lat / 1000, 2), "км")

Синий цвет - ДТП

Оранжевый - КФВФ

Зеленый - знак ограничения скорости

In [ ]:
m

In [ ]:
strom_street = pd.DataFrame(data=q, columns=['количество_дтп', 'количество_знаков_кфвф', 'количество_знаков_ограничения_скорости'])
strom_street

In [ ]:
distance_street_strom

# Проверка гипотезы

In [ ]:
all = pd.concat([strom_street, kut_pros])


**стат тест**

In [ ]:
df=all

In [ ]:
df = df[df.количество_знаков_ограничения_скорости > 0].copy()

In [ ]:
df['камера_в_кластере'] = df['количество_знаков_кфвф'].apply(lambda x: 'да' if x > 0 else 'нет')

from scipy.stats import ttest_ind

# разделение данных на две группы в зависимости от наличия камеры в кластере
group1 = df[df['камера_в_кластере'] == 'нет']['количество_дтп']
group2 = df[df['камера_в_кластере'] == 'да']['количество_дтп']

# проведение t-теста
t, p = ttest_ind(group1, group2)

# вывод результатов теста
print("t-статистика:", t)
print("p-значение:", p)

if p < 0.05:
    print("Гипотеза подтверждена")
else:
    print("Гипотеза не подтверждена")


Значение t-статистики показывает, насколько отличаются средние значения количества ДТП на участках дорог с камерами и без камер. Значение t-статистики отрицательное, что говорит о том, что на участках дорог с камерами количество ДТП меньше, чем на участках дорог без камер.

Исходя из полученных результатов теста, можно сделать вывод о том, что гипотеза подтверждена на выбранном уровне значимости. То есть, на участках автомобильных дорог, оборудованных и знаком ограничения скорости, и знаком о работе камер фото- и видеофиксации, действительно регистрируется меньшее число ДТП, чем на участках автомобильных дорог, оборудованных только знаками ограничения скорости. Это может свидетельствовать о том, что применение камер фото- и видеофиксации способствует снижению количества ДТП на дорогах.

# Доказательство с помощью моделирования

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
df = all

# df = df[df.количество_знаков_ограничения_скорости > 0].copy()
df['знак_кфвф_в_кластере'] = df['количество_знаков_кфвф'].apply(lambda x: '1' if x > 0 else '0')
df['факт_дтп'] = df['количество_дтп'].apply(lambda x: '1' if x > 0 else '0')
df['знак_огр_скорости'] = df['количество_знаков_ограничения_скорости'].apply(lambda x: '1' if x > 0 else '0')

df['факт_дтп'] = df['факт_дтп'].astype('int64')


# разделяем данные на тренировочный и тестовый наборы
train_data = df.sample(frac=0.7, random_state=42)
test_data = df.drop(train_data.index)

# определяем целевую переменную и признаки
target = 'факт_дтп'
features = ['количество_знаков_кфвф', 'количество_знаков_ограничения_скорости', 'знак_кфвф_в_кластере', 'знак_огр_скорости']

посмотрим на баланс классов

In [ ]:
df.факт_дтп.value_counts()

# Дерево решений + случайный лес + градиентный бустинг из коробки

Нижеуказанный код строит четыре различных модели машинного обучения и обучает каждую на обучающих данных, затем тестирует их на тестовых данных, используя различные метрики для оценки их производительности.

Модели, используемые в этом коде, включают решающее дерево, случайный лес, логистическую регрессию и CatBoost. Для каждой модели вычисляются показатели точности, точности, полноты, F1-оценки и ROC AUC-оценки, которые затем сохраняются в DataFrame.

In [ ]:


# список моделей и их параметров
models = [{'name': 'DecisionTreeClassifier', 'model': DecisionTreeClassifier(random_state=42)},
          {'name': 'RandomForestClassifier', 'model': RandomForestClassifier(random_state=42, class_weight='balanced')},
          {'name': 'LogisticRegression', 'model': LogisticRegression(random_state=42, class_weight = 'balanced')},
          {'name': 'CatBoostClassifier', 'model': CatBoostClassifier(random_state=42, verbose=False)}]

# список для хранения метрик каждой модели
metrics = []

# обучаем каждую модель, делаем предсказания на тестовых данных и вычисляем метрики
for m in models:
    model = m['model']
    model.fit(train_data[features], train_data[target])
    predictions = model.predict(test_data[features])
    accuracy = accuracy_score(test_data[target], predictions)
    precision = precision_score(test_data[target], predictions)
    recall = recall_score(test_data[target], predictions)
    f1 = f1_score(test_data[target], predictions)
    roc_auc = roc_auc_score(test_data[target], predictions)
    metrics.append({'Model': m['name'], 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1, 'ROC AUC Score': roc_auc})

# создаем датафрейм с метриками для всех моделей
metrics_df = pd.DataFrame(metrics)

metrics_df


Это таблица, в которой перечислены метрики для каждой модели машинного обучения, которая была обучена на данных для проверки гипотезы. Каждая метрика характеризует качество предсказаний модели, при оценке точности, полноты, F1-меры и ROC-AUC-меры.

Модели обучались на признаках (features) и целевой переменной (target) для проверки гипотезы о том, что на участках дорог с установленными знаками ограничения скорости и работающими камерами фото- и видеофиксации регистрируется меньшее число ДТП, чем на участках дорог только с знаками ограничения скорости.

Вывод: Метрики, полученные для моделей машинного обучения, могут помочь в проверке гипотезы, но они не являются единственным индикатором ее подтверждения или опровержения.
Однако, если рассмотреть метрики, полученные для моделей, можно сделать следующие выводы:

- Модели DecisionTreeClassifier и CatBoostClassifier показали показатель recall = 1.0, что означает, что они правильно определяют все положительные случаи. Это может свидетельствовать о том, что на участках дорог, оборудованных и знаком ограничения скорости, и знаком о работе камер фото- и видеофиксации, регистрируется меньшее число ДТП.

- Однако, в то же время, значение ROC AUC Score у моделей DecisionTreeClassifier и CatBoostClassifier равно 0.5, что может свидетельствовать о том, что модели не могут отличить между собой положительные и отрицательные классы. Таким образом, метрики не дают однозначного ответа на вопрос о подтверждении или опровержении гипотезы.

- Другие модели, такие как RandomForestClassifier и LogisticRegression, показали низкие значения метрик, таких как recall, precision, и F1 Score, что может свидетельствовать о том, что они плохо определяют положительные случаи, т.е. на участках автомобильных дорог, оборудованных и знаком ограничения скорости, и знаком о работе камер фото- и видеофиксации, регистрируется меньшее число ДТП.

Таким образом, метрики показывают, что гипотеза может быть верна, учитывая показатели на t-тесте